# Ground Truth Creation for Gabor Embeddings


### Imports     

In [45]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse import save_npz
from scipy.sparse import load_npz
import math

pd.set_option('display.max_colwidth', None)

### load dataset

In [53]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'no_product_variants.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=10, random_state=2).reset_index(drop=True)
display(product_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,38.0,schwarz,05.160.37,6590677,315573,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0
2,547193,4054452768427,Slipper Glattleder schwarz,42.0,schwarz,04.443.27,6590677,315576,w,SK,...,Halbschuhe,Damenschuhe,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0
3,549776,4059701687894,Sneaker low Rauleder blau,39.0,blau,06.968.46,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat schwarz,37.5,schwarz,06.102.67,6590678,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,35.0,blauw,24.764.36,6590677,315568,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,40.5,pink,26.090.21,6590678,315590,w,PT,...,Halbschuhe,Damenschuhe,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0
3294,10782704,4251234499207,Shopper ANDIE blau,,blau,921453,363013,363017,w,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3295,10679703,4066558303617,Elegante pumps Glad leer wit,35.0,wit,21.450.60,6590677,315573,w,PT,...,Pumps,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0


### Encode data

In [54]:
product_data = product_data.drop(['productNumber', 'productName', 'productSizeEu', 'articleNumber', 'productType_y', 'heelHeightGroup', 'sizeEu', 'sizeIndex', 'sizeUk', 'shaftLength__value', 'isTransferee', 'isSuccessor', 'productSizeEu', 'b2cPrice', 'b2cCurrency', 'indicatorAdvertisement', 'indicatorAdvertisement', 'indicatorWedding', 'indicatorSale', 'heelHeight__unit', 'heelHeight__value', 'width', 'width__unit', 'width__value', 'hasOwnAssortment', 'formGroup', 'model', 'specialOutfits', 'weight__unit', 'weight__value', 'o_id_y', 'fieldname_y', 'fieldname_x', 'heelForm', 'liningPm', 'height__unit', 'height__value', 'length__value', 'length__unit', 'productType_x', 'o_id_x', 'materialPm'], axis=1)
display(product_data)

,product__id,productColorName,brand__id,mainCategory__id,gender,originCountry,line,material,color,style,shoeTip,lining,shoeType,shoeStyle,sole
0,557559,schwarz,6590678,315571,w,PT,F-S|H-W,Glattleder,schwarz,SPC,rund,Microfutter,sportliche Ballerinas,Ballerinas,Gummi
1,551622,schwarz,6590677,315573,w,PT,F-S|H-W,Glattleder,schwarz,ELK,karree,Lederfutter,elegante Pumps,Pumps,EVA
2,547193,schwarz,6590677,315576,w,SK,F-S|H-W,Glattleder,schwarz,SPC,karree,Textilfutter,Slipper,Halbschuhe,PU-TPU
3,549776,blau,4997827,315567,w,VN,F-S|H-W,Rauleder,blau,SPC,rund,Lederfutter,Sneaker low,Sneaker,Gummi-EVA
4,550726,schwarz,6590678,315574,w,SK,F-S|H-W,Materialmix Lederimitat,schwarz,ELK,karree,Microfutter,elegante Ballerinas,Ballerinas,EVA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,blauw,6590677,315568,w,SK,F-S,Rauleder,blau,ELK,rund,Lederfutter,Plateau Sandalen,Sandalen,PU-TPU
3293,10442948,pink,6590678,315590,w,PT,F-S,Materialmix Leder,pink,ELK,rund,Lederfutter,Mokassins,Halbschuhe,Gummi
3294,10782704,blau,363013,363017,w,DE,F-S,Kunstleder,blau,NaN,NaN,NaN,NaN,NaN,NaN
3295,10679703,wit,6590677,315573,w,PT,F-S,Glattleder,weiß,ELK,spitz,Lederfutter,elegante Pumps,Pumps,TPU


In [55]:

cols_to_encode = {
    'gender': 0.2, 
    'brand__id': 0.2, 
    'color': 0.1, 
    'mainCategory__id': 0.1, 
    'shoeStyle': 0.1, 
    'productColorName': 0.05, 
    'sole': 0.05, 
    'line': 0.05, 
    'lining': 0.05,
    'material' : 0.03, 
    'style': 0.03, 
    'shoeType': 0.02, 
    'shoeTip': 0.01, 
    'originCountry': 0.01
}
count = 0
for col in cols_to_encode:
    product_data[col + '_code'] = pd.factorize(product_data[col])[0]
    product_data = product_data.drop([col], axis=1)
    count += cols_to_encode[col]
display(product_data, count)

,product__id,gender_code,brand__id_code,color_code,mainCategory__id_code,shoeStyle_code,productColorName_code,sole_code,line_code,lining_code,material_code,style_code,shoeType_code,shoeTip_code,originCountry_code
0,557559,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,551622,0,1,0,1,1,0,1,0,1,0,1,1,1,0
2,547193,0,1,0,2,2,0,2,0,2,0,0,2,1,1
3,549776,0,2,1,3,3,1,3,0,1,1,0,3,0,2
4,550726,0,0,0,4,0,0,1,0,0,2,1,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,0,1,1,36,4,30,2,1,1,1,1,33,0,1
3293,10442948,0,0,18,33,2,56,0,1,1,6,1,30,0,0
3294,10782704,0,4,1,46,-1,1,-1,1,-1,12,-1,-1,-1,5
3295,10679703,0,1,2,1,1,33,4,1,1,0,1,1,2,0


1.0000000000000002

In [56]:
count

1.0000000000000002

### Create Ground Truth


In [73]:
def calculate_similarity(item1, item2, print_results=False):

    similarity_scores = []
    for col in cols_to_encode:
        if item1[col + '_code'] == item2[col + '_code']:
            similarity = 1  # Items are the same
        else:
            similarity = 0  # Items are different
        # weighted_similarity = similarity * cols_to_encode[col]
        similarity_scores.append(similarity)
    overall_similarity = np.average(similarity_scores, weights=list(cols_to_encode.values()))

    return overall_similarity

def combinations(n, r):
    return math.factorial(n) // (math.factorial(r) * math.factorial(n - r))

def weighted_average(value1, weight1, value2, weight2):
    return (value1 * weight1 + value2 * weight2) / (weight1 + weight2)

In [74]:
search_1_idx = 0
search_2_idx = 1
display(calculate_similarity(product_data.iloc[search_1_idx], product_data.iloc[search_2_idx], False))

0.43999999999999995

In [77]:
treshold = 0.7
df = product_data.copy()
print_count = 1000
similarity_matrix = lil_matrix((product_data.shape[0], product_data.shape[0]), dtype=bool)  # Sparse matrix
for i in df.index:
    for j in range(i + 1, df.shape[0]):
        similarity_matrix[i, j] =  calculate_similarity(df.iloc[i], df.iloc[j]) >= treshold
    if i+1 % print_count == 0:
        print("done: " + str(i+1) + "/" + str(df.shape[0]))
print(str(similarity_matrix.count_nonzero()) + " true pairs from possible " + str(combinations(df.shape[0], 2)))

169014 true pairs from possible 5433456


In [78]:
similarity_matrix_csr = similarity_matrix.tocsr()

In [79]:
filename = 'ground_truth/ground_truth_no_variants.npz'
save_npz(basepath + filename, similarity_matrix_csr)

In [80]:
loaded_csr = load_npz(basepath + filename)
display(loaded_csr[0,1])

False